#**LangChain**

LangChain is a framework for developing applications powered by language models.

- GitHub: https://github.com/hwchase17/langchain
- Docs: https://python.langchain.com/v0.2/docs/introduction/

### Overview:
- Installation
- LLMs
- Prompt Templates
- Chains
- Agents and Tools
- Memory
- Document Loaders
- Indexes

#**01: Installation**

In [ ]:
!pip install langchain langchain_community

#**02: Setup the Environment**

In [ ]:
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = ""

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

##**03: Large Language Models**

The basic building block of LangChain is a Large Language Model which takes text as input and generates more text

Suppose we want to generate a company name based on the company description, so we will first initialize an OpenAI wrapper. In this case, since we want the output to be more random, we will intialize our model with high temprature.

The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

temperature value--> how creative we want our model to be

0 ---> temperature it means model is  very safe it is not taking any bets.

1 --> it will take risk it might generate wrong output but it is very creative

A generic interface for all LLMs. See all LLM providers: https://python.langchain.com/en/latest/modules/models/llms/integrations.html

#**Open AI**

#**Example 1**

In [ ]:
!pip install openai

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)

And now we will pass in text and get  predictions

In [ ]:
text="What would be a good company name for a company that makes colorful socks?"

In [ ]:
print(llm.predict(text))

In [ ]:
print(llm(text))

In [ ]:
print(llm.invoke(text))

#**Example 2**

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Chinese food. Suggest a fency name for this.")
print(name)

In [ ]:
response=llm("I want to open a restaurant for Chinese food. Suggest a fency name for this.")
print(response)

#**Hugging Face**

#**Example 1**

In [ ]:
!pip install huggingface_hub

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
# https://huggingface.co/google/flan-t5-xl

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})

llm("translate English to German: How old are you?")

#**Example 2**

In [ ]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})
# name = llm.predict("I want to open a restaurant for Chinese food. Suggest a fency name for this.")
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

##**04: Prompt Templates**

Currently in the above applications we are writing an entire prompt, if you are creating a user directed application then this is not an ideal case

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In many Large Language Model applications we donot pass the user input directly to the Large Language Model, we add the user input to a large piece of text called prompt template

#**Example 1**

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="indian")
print(p)

#**Example 2**

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

##**05: Chains**

Combine LLMs and Prompts in multi-step workflows

Now as we have the  **model**:


  llm = OpenAI(temperature=0.9)


and the **Prompt Template**:

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")


prompt.format(product="colorful socks")


Now using Chains we will link together model and the PromptTemplate and other Chains

The simplest and most common type of Chain is LLMChain, which passes the input first to Prompt Template and then to Large Language Model

LLMChain is responsible to execute the PromptTemplate, For every PromptTemplate we will specifically have an LLMChain

#**Example 1**

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

Whatever input text i am giving that will get assigned to this particular variable that is **product**

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
response= chain.run("colorful socks")
print(response)

#**Example 2**

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
response=chain.run("Mexican")
print(response)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
response=chain.run("Mexican")
print(response)

**Can we combine Multiple PromptTemplates, We will try to combine Multiple PromptTemplates**

**The output from the first PromptTemplate is passed to the next PromptTemplate as input**

#**To combine the Chain and  to set a sequence for that we use SimpleSequentialChain**

##**Simple Sequential Chain**

In [ ]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("indian")
print(content)

**There is a issue with SimpleSequentialChain it only shows last input information**

#**To show the entire information i will use SequentialChain**

##**Sequential Chain**

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [ ]:
print(chain({"cuisine": "indian"}))

##**06. Agents and Tools**

Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.


When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
- LLM: The language model powering the agent.
- Agent: The agent to use.


Agent is a very powerful concept in LangChain

For example I have to travel from Dubai to Canada, I type this in ChatGPT



---> Give me  two flight options from Dubai to Canada on September 1, 2024 | ChatGPT will not be able to answer because has knowledge till
September 2021



ChatGPT plus has Expedia Plugin, if we enable this plugin it will go to Expedia Plugin and will try to pull information about Flights & it will show the information

SerpApi is a real-time API to access Google search results.

#### Wikipedia and llm-math tool

In [ ]:
!pip install wikipedia

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Let's test it out!


agent.run("What was the GDP of US in 2024?")

##**07: Memory**

Chatbot application like ChatGPT, you will notice that it remember past information

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)

In [ ]:
name = chain.run("Indian")
print(name)

In [ ]:
chain.memory

In [ ]:
type(chain.memory)

##**ConversationBufferMemory**

We can attach memory to remember all previous conversation

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

In [ ]:
name = chain.run("Arabic")
print(name)

In [ ]:
print(chain.memory.buffer)

##**ConversationChain**

Conversation buffer memory goes growing endlessly

Just remember last 5 Conversation Chain

Just remember last 10-20 Conversation Chain

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

In [ ]:
convo.run("Who won the first cricket world cup?")

In [ ]:
convo.run("How much is 5+5?")

In [ ]:
convo.run("Who was the captain of the winning team?")

In [ ]:
print(convo.memory.buffer)

##**ConversationBufferWindowMemory**

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

In [ ]:
convo.run("How much is 5+5?")

In [ ]:
convo.run("Who was the captain of the winning team?")

In [ ]:
print(convo.memory.buffer)

#**08: Document Loaders**


In [ ]:
!pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/my_paper.pdf")
pages = loader.load()

In [ ]:
pages